<a href="https://colab.research.google.com/github/krystaldowling/MAST30034_Final_Project-/blob/CNN_Model/Standard_CNN_GridSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This document contains code modified from https://realpython.com/python-keras-text-classification/. 

In [1]:
# Import statements
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [2]:
# Loading in preproccessed data
PATH = "/content/drive/My Drive/Data/"
data = pd.read_csv(PATH + "preproccessed_data.csv", lineterminator='\n')

data

,title,text,label
0,muslims busted stole millions gov ’ benefits,print pay back money plus interest entire fami...,1
1,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1
2,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1
3,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1
4,fantastic trumps 7 point plan reform healthcar...,email healthcare reform make america great sin...,1
...,...,...,...
29449,process analytical instruments market – techna...,technavio published new report global process ...,0
29450,travel deals get 1200 air credit two apt cookt...,apt offering savings new cape york outback wil...,0
29451,taiwanese recyclers belief waste simply mispla...,taipei taiwan sept 8 2015 prnewswire recent ye...,0
29452,season curtain raiser ideal way honour john,blackburn sunday league john haydock memorial ...,0


In [3]:
# Performing test-train split
text_train, text_test, y_train, y_test = train_test_split(data['text'].values, data['label'].values, test_size=0.25, random_state=1000)

In [4]:
# calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])
 
maxlen = max_length(data['text'])

In [5]:
# Tokenizing data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_test = tokenizer.texts_to_sequences(text_test)

# Vocab size- adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1  

In [6]:
# Padding sequences
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [7]:
# Create Model
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(6, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [8]:
# Defining possible parameters 
param_grid = dict(num_filters=[32, 64, 128],
                  kernel_size=[3, 5, 7],
                  vocab_size= [vocab_size], 
                  embedding_dim=[100],
                  maxlen= [maxlen])

In [10]:
# Run grid search on data
model = KerasClassifier(build_fn=create_model, epochs=6, batch_size=16)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, cv=4, verbose=1, n_iter=4)
# Fitting Model
grid_result = grid.fit(X_train, y_train)

# Evaluate testing set
test_accuracy = grid.score(X_test, y_test)
print(test_accuracy)


Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/6
1036/1036 [==============================] - 193s 187ms/step - loss: 0.3972 - accuracy: 0.8195
Epoch 2/6
1036/1036 [==============================] - 193s 186ms/step - loss: 0.2066 - accuracy: 0.9262
Epoch 3/6
1036/1036 [==============================] - 192s 186ms/step - loss: 0.0801 - accuracy: 0.9773
Epoch 4/6
1036/1036 [==============================] - 192s 185ms/step - loss: 0.0234 - accuracy: 0.9956
Epoch 5/6
1036/1036 [==============================] - 191s 184ms/step - loss: 0.0135 - accuracy: 0.9976
Epoch 6/6
346/346 [==============================] - 2s 5ms/step - loss: 0.5558 - accuracy: 0.8682
Epoch 1/6
1036/1036 [==============================] - 192s 185ms/step - loss: 0.3811 - accuracy: 0.8284
Epoch 2/6
1036/1036 [==============================] - 191s 184ms/step - loss: 0.2027 - accuracy: 0.9258
Epoch 3/6
1036/1036 [==============================] - 190s 184ms/step - loss: 0.0824 - accuracy: 0.9754
Epoch 4/6
1036/1036 [==============================] - 189s 1

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed: 293.5min finished


1381/1381 [==============================] - 234s 169ms/step - loss: 0.3584 - accuracy: 0.8441
Epoch 2/6
1381/1381 [==============================] - 233s 169ms/step - loss: 0.2078 - accuracy: 0.9254
Epoch 3/6
1381/1381 [==============================] - 233s 169ms/step - loss: 0.1195 - accuracy: 0.9614
Epoch 4/6
1381/1381 [==============================] - 233s 169ms/step - loss: 0.0740 - accuracy: 0.9761
Epoch 5/6
1381/1381 [==============================] - 234s 169ms/step - loss: 0.0391 - accuracy: 0.9883
Epoch 6/6
461/461 [==============================] - 2s 4ms/step - loss: 0.5425 - accuracy: 0.8725
0.8724877834320068


In [11]:
# Save and evaluate results
s = ('Running data set\nBest Accuracy : {:.4f}\n{}\nTest Accuracy : {:.4f}\n\n')
output_string = s.format( grid_result.best_score_, grid_result.best_params_, test_accuracy)
print(output_string)

Running data set
Best Accuracy : 0.8725
{'vocab_size': 192727, 'num_filters': 64, 'maxlen': 2202, 'kernel_size': 3, 'embedding_dim': 100}
Test Accuracy : 0.8725




In [12]:
# Using code modified from https://machinelearningmastery.com/how-to-calculate-precision-recall-f1-and-more-for-deep-learning-models/

pred = grid.predict(X_test)
pred = np.rint(pred)

# Precision Score
precision = precision_score(y_test, pred)
print('Test Precision: %f' % (precision*100))

# Recall Score
recall = recall_score(y_test, pred)
print('Test Recall: %f' % (recall*100))

# F1 Score
f1 = f1_score(y_test, pred)
print('Test F1 score: %f' % (f1*100))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Test Precision: 87.006961
Test Recall: 82.755359
Test F1 score: 84.827921
